### Notebook objectives
In this notebook we plot the output of three different of fit method
1 - Grid fit (simple GLM with different grid steps)
2 - Iterative fit wihtout bound 
3 - Iterative fit with predefined bound

In [1]:
import sys, os
import numpy as np
import glob
import datetime
import json
import ipdb
import plotly.graph_objects as go
import random
import importlib
import cortex
import pandas as pd

import sys
sys.path.append("{}/../../../../utils".format(os.getcwd()))
from pycortex_utils import set_pycortex_config_file
from surface_utils import make_surface_image , load_surface

In [ ]:
# data filenames and folder
subject = 'sub-02'
rois = ['occ', 'par', 'frt']

main_dir = '/home/ulascombes/disks/meso_shared'
#main_dir = '/Users/uriel/disks/meso_shared'
#main_dir = '/home/mszinte/disks/meso_S/data/'
project_dir = 'RetinoMaps'
subject = 'sub-02'
gii_file = 'sub-02_task-pRF_hemi-L_fmriprep_dct_avg_bold.func.gii'

cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
data_path = '{}/{}/derivatives/pp_data/sub-02/prf/fit/test'.format(main_dir, project_dir)
input_fn_fsnative = '{}/{}/derivatives/pp_data/{}/func/fmriprep_dct_avg/fsnative/{}'.format(
    main_dir, project_dir, subject, gii_file)
prf_fit_test_dir = "{}/{}/derivatives/pp_data/{}/prf/fit/test".format(
    main_dir, project_dir, subject)

In [ ]:
set_pycortex_config_file(cortex_dir)
importlib.reload(cortex)

In [ ]:
# load arrays
gauss_fit_mat_repeat = np.load(file='{}/{}_gauss_fit_mat_repeat.npy'.format(
    prf_fit_test_dir, subject))
gauss_pred_mat_repeat = np.load(file='{}/{}_gauss_pred_mat_repeat.npy'.format(
    prf_fit_test_dir, subject))
gauss_duration_repeat = np.load(file='{}/{}_gauss_duration_repeat.npy'.format(
    prf_fit_test_dir, subject))

# Load fsnative data 
data_img_fsnative, data_fsnative = load_surface(input_fn_fsnative)



In [ ]:
# Get rois
roi_verts = cortex.get_roi_verts(subject=subject, 
                                 roi=rois, 
                                 mask=True)

In [ ]:
# Get corresponding vertices
vert_mask_occ = roi_verts['occ']
vert_num_occ = np.sum(vert_mask_occ)
vert_mask_par = roi_verts['par']
vert_num_par = np.sum(vert_mask_par)
vert_mask_frt = roi_verts['frt']
vert_num_frt = np.sum(vert_mask_frt)

vert_mask = dict(occ=[0, vert_num_occ],
                 par=[vert_num_occ, vert_num_occ + vert_num_par],
                 frt=[vert_num_occ + vert_num_par, vert_num_occ + vert_num_par + vert_num_frt])


# number of repeat of fit
n_repeat = gauss_fit_mat_repeat.shape[2]-2
grid_idx = 0
r2_idx = -1
without_bound_idx = 1
with_bound_idx = np.arange(2,n_repeat+2,1)

In [ ]:
# figures settings
colors =['rgb(242, 183, 1)',
         'rgb(57, 105, 172)',
         'rgb(17, 165, 121)',
         'rgb(127, 60, 141)']

linewidth = 2


# Figure 1

In [ ]:
fig = go.Figure()

for i , roi in enumerate(rois) :
    
    # select roi data 
    roi_data = gauss_fit_mat_repeat[vert_mask[roi][0]:vert_mask[roi][1]:,:,:]

    # plot iterative fit with bound
    fig.add_trace(go.Bar(x=np.arange(0,n_repeat,1)+1,
                         y=np.mean(roi_data[:,r2_idx,with_bound_idx], axis=0),
                         marker_color=colors[i],
                         marker_line_color=colors[i],
                         marker_line_width = 3,
                         error_y_array = np.std(roi_data[:,r2_idx,with_bound_idx], axis=0),
                         error_y_symmetric=False,
                         error_y_arrayminus = np.std(roi_data[:,r2_idx,with_bound_idx], axis=0)*0,
                         error_y_color = colors[i],
                         name='Iterative fit with bound {}'.format(roi),
                         showlegend=True,
                         width = 0.2))
    
    # plot iterative fit without bound
    xplot=np.arange(0,n_repeat+2,1)
    fig.add_trace(go.Scatter(x=xplot,
                             y=xplot*0+np.mean(roi_data[:,r2_idx,without_bound_idx], axis=0),
                             mode='lines',
                             line=dict(color=colors[i], width=linewidth, dash='dot'),
                             showlegend=False,
                            ))

    # plot grid fit
    xplot=np.arange(0,n_repeat+2,1)
    fig.add_trace(go.Scatter(x=xplot,
                             y=xplot*0+np.mean(roi_data[:,r2_idx,grid_idx], axis=0),
                             mode='lines',
                             line=dict(color=colors[i], width=linewidth, dash='solid'),
                             showlegend=False,
                            ))

    
# for the legend
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color='black', width=linewidth, dash='dot'),
                         name='Iterative fit without bound'))

fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                          line=dict(color='black', width=linewidth, dash='solid'),
                          name='Grid fit'))


# figure layout
fig.update_layout(template='simple_white',
                  title = 'R<sup>2</sup> as a function of iterative fit repetitions',
                  xaxis_title = 'Iterative fit repetition',
                  yaxis_title = 'R<sup>2</sup>'
                 )

fig.update_xaxes(range=[0,n_repeat+1])
fig.update_yaxes(range=[0,0.8])
fig.show()

# Figure 2

In [ ]:
fig2 = go.Figure()

# plot duration of iterative with bound
fig2.add_trace(go.Bar(x=np.arange(0,n_repeat,1)+1,
                     y=gauss_duration_repeat[with_bound_idx],
                     marker_color='gray',
                     marker_line_color='gray',
                     marker_line_width = 3,
                     name='Iterative fit with bound',
                     showlegend=True,
                     width = 0.2))

# plot iterative fit without bound
xplot=np.arange(0,n_repeat+2,1)
fig2.add_trace(go.Scatter(x=xplot,
                          y=xplot*0+gauss_duration_repeat[without_bound_idx],
                          mode='lines',
                          line=dict(color='gray', width=linewidth, dash='dot'),
                          name='Iterative fit without bound',
                          showlegend=True,
                         ))

# plot grid fit
fig2.add_trace(go.Scatter(x=xplot,
                          y=xplot*0+gauss_duration_repeat[grid_idx],
                          mode='lines',
                          line=dict(color='gray', width=linewidth, dash='solid'),
                          name='Grid fit ',
                          showlegend=True,
                         ))

fig2.update_layout(template='simple_white',
                  title = 'Durations of the different type of fits',
                  xaxis_title = 'Iterative fit repetition',
                  yaxis_title = 'Duration (s)')
fig2.update_xaxes(range=[0,n_repeat+1])
fig2.update_yaxes(range=[0,1500])

fig2.show()

In [ ]:
gauss_duration_repeat[without_bound_idx]

# Figure 3

In [ ]:
fig3 = go.Figure()
cumulative_durations = np.cumsum(gauss_duration_repeat[2:])

# plot duration of iterative with bound
fig3.add_trace(go.Bar(x=np.arange(0,n_repeat,1)+1,
                     y=np.cumsum(gauss_duration_repeat[with_bound_idx]),
                     marker_color='gray',
                     marker_line_color='gray',
                     marker_line_width = 3,
                     name='Iterative fit with bound',
                     showlegend=True,
                     width = 0.2))

# plot iterative fit without bound
xplot=np.arange(0,n_repeat+2,1)
fig3.add_trace(go.Scatter(x=xplot,
                          y=xplot * 0 + np.cumsum(gauss_duration_repeat[without_bound_idx]),
                          mode='lines',
                          line=dict(color='gray', width=linewidth, dash='dot'),
                          name='Iterative fit without bound',
                          showlegend=True,
                         ))

# plot grid fit
fig3.add_trace(go.Scatter(x=xplot,
                          y=xplot * 0 + np.cumsum(gauss_duration_repeat[grid_idx]),
                          mode='lines',
                          line=dict(color='gray', width=linewidth, dash='solid'),
                          name='Grid fit ',
                          showlegend=True,
                         ))

fig3.update_layout(template='simple_white',
                  title = 'Durations of the different type of fits',
                  xaxis_title = 'Iterative repetition',
                  yaxis_title = 'Cumulative duration (s)')

fig3.update_xaxes(range=[0,n_repeat+1])
fig3.update_yaxes(range=[0,1500])

fig3.show()

# Figure 4

In [ ]:
max_r2 = np.argmax(gauss_fit_mat_repeat[:,-1,-1])
num_vert = max_r2

fig4 = go.Figure()
# plot data time series
fig4.add_trace(go.Scatter(y=data_fsnative[:,num_vert],
                         name='data', 
                         mode='markers'))

# plot last iterativ repeat prediction
fig4.add_trace(go.Scatter(y=gauss_pred_mat_repeat[num_vert,:,n_repeat],
                         name='Last iterative fit with bounds (R2={:1.2f})'.format(gauss_fit_mat_repeat[num_vert,r2_idx,n_repeat])))

# plot grid prediction
fig4.add_trace(go.Scatter(y=gauss_pred_mat_repeat[num_vert,:,grid_idx],
                         name='Grid search results (R2={:1.2f})'.format(gauss_fit_mat_repeat[num_vert,r2_idx,grid_idx])))

# plot Iterativ no bounds prediction
fig4.add_trace(go.Scatter(y=gauss_pred_mat_repeat[num_vert,:,without_bound_idx],
                         name='Iterative fit without bounds (R2={:1.2f})'.format(gauss_fit_mat_repeat[num_vert,r2_idx,without_bound_idx])))


fig4.update_layout(template='simple_white',
                  title = 'Fit predictions',
                  xaxis_title = 'TR (sec)',
                  yaxis_title = 'z-score')
fig4.update_xaxes(range=[0,data_fsnative.shape[0]])
fig4.update_yaxes(range=[-4,4])

fig4.show()

# Figure 5 

In [ ]:
# make a df 
dic = {'r2': [], 'rois': [], 'model': []}
df1 = pd.DataFrame(dic)
df2 = pd.DataFrame(dic)
df3 = pd.DataFrame(dic)


for roi in rois:
    r2_values = gauss_fit_mat_repeat[vert_mask[roi][0]:vert_mask[roi][1], r2_idx, grid_idx]
    r2_values_df = pd.DataFrame(r2_values, columns=['r2'])  
    roi_list = [roi] * len(r2_values)
    roi_df = pd.DataFrame({'rois': roi_list})
    roi_df['model'] = 'Grid search results'
    df1 = pd.concat([df1, pd.concat([roi_df, r2_values_df], axis=1)], ignore_index=True)

for roi in rois:
    r2_values = gauss_fit_mat_repeat[vert_mask[roi][0]:vert_mask[roi][1], r2_idx, without_bound_idx]
    r2_values_df = pd.DataFrame(r2_values, columns=['r2'])  
    roi_list = [roi] * len(r2_values)
    roi_df = pd.DataFrame({'rois': roi_list})
    roi_df['model'] = 'Last iterative fit without bounds'
    df2 = pd.concat([df2, pd.concat([roi_df, r2_values_df], axis=1)], ignore_index=True)

for roi in rois:
    r2_values = gauss_fit_mat_repeat[vert_mask[roi][0]:vert_mask[roi][1], r2_idx, -1]
    r2_values_df = pd.DataFrame(r2_values, columns=['r2'])  
    roi_list = [roi] * len(r2_values)
    roi_df = pd.DataFrame({'rois': roi_list})
    roi_df['model'] = 'Last iterative fit with bounds'
    df3 = pd.concat([df3, pd.concat([roi_df, r2_values_df], axis=1)], ignore_index=True)
    
df_final = pd.concat([df1, df2, df3])


In [ ]:
fig5 = go.Figure()

models=["Last iterative fit with bounds","Grid search results", "Last iterative fit without bounds"]


for j, model in enumerate(models):
    df_type = df_final.loc[df_final.model == model]
    
    fig5.add_trace(go.Box(
        x=df_type.rois,
        y=df_type.r2,
        legendgroup=model,
        name=model,
        showlegend=False,
        notched=False,
        boxpoints='all',
        jitter=0.5,
        pointpos=0,
        line_color='rgba(0,0,0,0)',
        fillcolor='rgba(0,0,0,0)',
        marker_color=colors[j],
        marker_opacity=0.4))

    fig5.add_trace(go.Box(
        x=df_type.rois,
        y=df_type.r2,
        legendgroup=model,
        name=model,
        showlegend=True,
        notched=False,
        boxpoints='all',
        jitter=1,
        pointpos=0,
        line_color=colors[j],
        fillcolor='rgba(0,0,0,0)',
        marker_color='rgba(0,0,0,0)'))
  
fig5.update_layout(boxmode='group',template='simple_white')
fig5.update_yaxes(title ='R<sup>2</sup>',range=[-0.1,1])
fig5.show()
               

# Figure 6

In [ ]:
fig6 = go.Figure()

models = ["Last iterative fit with bounds", "Grid search results", "Last iterative fit without bounds"]

for j, model in enumerate(models):
    df_type = df_final[df_final['model'] == model]

    fig6.add_trace(go.Violin(
        x=df_type.rois,  
        y=df_type.r2,
        legendgroup=model,
        name=model,
        line_color=colors[j],
        bandwidth=0.08,
        fillcolor='rgba(0,0,0,0)',
        marker_color='rgba(0,0,0,0)',
        box_visible=True, 
        ))




fig6.update_layout(template='simple_white' ,violinmode='group')
fig6.update_yaxes(title='R<sup>2</sup>', range=[-0.3, 1.2])

fig6.show()
